# Model Evaluation Notebook

Interactive notebook to evaluate trained models:
- Select model from dropdown
- Automatically load corresponding config
- Build complete dataset (start → test_end)
- Load model and generate predictions

**Model Naming Convention:**
- `cl_m-tft_out-48_freq-1h_wind_50_static.pt` → `config_wind_50.yaml` (with static features)
- `cl_m-tft_out-48_freq-1h_wind_50_nostatic.pt` → `config_wind_50.yaml` (no static features)
- `cl_m-tft_out-48_freq-1h_wind_00164.pt` → `config_wind_00164.yaml`

## 1. Imports and Setup

In [31]:
import os
import re
import gc
import yaml
import torch
import numpy as np
import pandas as pd
from pathlib import Path
from tqdm import tqdm
import logging
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

# Interactive widgets
import ipywidgets as widgets
from IPython.display import display, clear_output

# Local utilities
from utils import preprocessing, tools, models, hpo

# Setup logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Set PyTorch settings
torch.set_float32_matmul_precision('high')

In [48]:
# =====================================
# HYPERPARAMETER SOURCE SELECTION
# =====================================
# Set this to control where hyperparameters are loaded from:
# - False: Use standard hyperparameters from config['model'] (default for non-HPO models)
# - True:  Load optimized hyperparameters from Optuna study (for HPO-trained models)

use_hpo_hyperparameters = False  # ← Change this to True if model was trained with HPO
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
target_key = 'synth_00164.csv'

print(f"Hyperparameter source: {'Optuna Study (HPO)' if use_hpo_hyperparameters else 'Config Defaults'}")

Hyperparameter source: Config Defaults


## 2. Helper Functions

In [49]:
def extract_config_name_from_model(model_filename):
    """
    Extract config name from model filename.

    Examples:
        'cl_m-tft_out-48_freq-1h_wind_50_static.pt' -> 'config_wind_50'
        'cl_m-tft_out-48_freq-1h_wind_00164.pt' -> 'config_wind_00164'

    Returns:
        config_name (str): Config filename without .yaml extension
        use_static (bool): True if '_static' suffix, False if '_nostatic', None otherwise
    """
    # Remove .pt extension
    name = model_filename.replace('.pt', '')

    # Check for static/nostatic suffix
    use_static = None
    if name.endswith('_static'):
        use_static = True
        name = name.replace('_static', '')
    elif name.endswith('_nostatic'):
        use_static = False
        name = name.replace('_nostatic', '')

    # Extract everything after '1h_'
    match = re.search(r'1h_(.*)', name)
    if match:
        config_suffix = match.group(1)
        config_name = f'config_{config_suffix}'
        return config_name, use_static
    else:
        raise ValueError(f"Could not extract config name from model filename: {model_filename}")


def load_config_for_model(model_filename, configs_dir='configs'):
    """
    Load config file for given model.

    Args:
        model_filename: Model filename (e.g., 'cl_m-tft_out-48_freq-1h_wind_50_static.pt')
        configs_dir: Directory containing config files

    Returns:
        config (dict): Loaded config
        use_static (bool): Whether to use static features
    """
    config_name, use_static = extract_config_name_from_model(model_filename)
    config_path = os.path.join(configs_dir, f'{config_name}.yaml')

    if not os.path.exists(config_path):
        raise FileNotFoundError(f"Config file not found: {config_path}")

    with open(config_path, 'r') as f:
        config = yaml.safe_load(f)

    logger.info(f"Loaded config: {config_path}")
    logger.info(f"Static features mode: {use_static}")

    return config, use_static


def adjust_static_features(config, use_static):
    """
    Adjust static features based on model suffix.

    Args:
        config: Config dictionary
        use_static: True to ensure static features, False to remove them, None to keep as-is

    Returns:
        Modified config
    """
    if use_static is None:
        # No suffix, use config as-is
        return config

    if use_static:
        # Ensure static features are present (already in config, just verify)
        logger.info(f"Using static features: {config.get('params', {}).get('static_features', [])}")
    else:
        # Remove static features
        if 'params' in config and 'static_features' in config['params']:
            config['params']['static_features'] = []
            logger.info("Removed static features (nostatic mode)")

    return config


print("✓ Helper functions defined")

✓ Helper functions defined


## 3. Interactive Model Selection

In [50]:
# Get all .pt model files
models_dir = 'models'
model_files = sorted([f for f in os.listdir(models_dir) if f.endswith('.pt')])

if not model_files:
    print(f"⚠️ No .pt model files found in {models_dir}/")
else:
    print(f"Found {len(model_files)} model(s):")
    for mf in model_files:
        print(f"  - {mf}")

# Create dropdown widget
model_dropdown = widgets.Dropdown(
    options=model_files,
    description='Select Model:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='600px')
)

# Display widget
display(model_dropdown)

Found 4 model(s):
  - cl_m-tft_out-48_freq-1h_wind_00164.pt
  - cl_m-tft_out-48_freq-1h_wind_02638.pt
  - cl_m-tft_out-48_freq-1h_wind_eight_nostatic.pt
  - cl_m-tft_out-48_freq-1h_wind_eight_static.pt


Dropdown(description='Select Model:', layout=Layout(width='600px'), options=('cl_m-tft_out-48_freq-1h_wind_001…

## 4. Load Config and Build Features

In [52]:
# Get selected model
selected_model = model_dropdown.value
print(f"Selected model: {selected_model}")

# Load corresponding config
config, use_static = load_config_for_model(selected_model)

# Adjust static features based on suffix
config = adjust_static_features(config, use_static)

# Handle frequency
config = tools.handle_freq(config=config)

# Set model name for preprocessing
model_name_match = re.search(r'm-([a-z]+)', selected_model)
if model_name_match:
    model_architecture = model_name_match.group(1)
    config['model']['name'] = model_architecture
    print(f"\n🏗️ Model architecture: {model_architecture.upper()}")
else:
    raise ValueError(f"Could not extract model architecture from filename: {selected_model}")

# Get features
features = preprocessing.get_features(config=config)

print("\n📋 Features configuration:")
print(f"  Known features ({len(features['known'])}): {features['known']}")
print(f"  Observed features ({len(features['observed'])}): {features['observed']}")
print(f"  Static features ({len(features['static'])}): {features['static']}")

2025-12-04 16:54:22,726 - INFO - Loaded config: configs/config_wind_eight.yaml
2025-12-04 16:54:22,727 - INFO - Static features mode: True
2025-12-04 16:54:22,728 - INFO - Using static features: ['park_age', 'cut_in', 'cut_out', 'rated_wind_speed', 'hub_height', 'rotor_diameter', 'altitude']


Selected model: cl_m-tft_out-48_freq-1h_wind_eight_static.pt

🏗️ Model architecture: TFT

📋 Features configuration:
  Known features (3): ['wind_speed_h78', 'wind_speed_h127', 'wind_speed_h184']
  Observed features (1): ['power']
  Static features (7): ['park_age', 'cut_in', 'cut_out', 'rated_wind_speed', 'hub_height', 'rotor_diameter', 'altitude']


## 5. Build Complete Dataset

Load raw data dictionary and process analog to `train_cl.py`.

In [53]:
print("📊 Building complete dataset...\n")

# Get data parameters
data_config = config.get('data', {})
test_end = pd.Timestamp(data_config.get('test_end'), tz='UTC')
freq = data_config.get('freq', '1h')
target_col = data_config.get('target_col', 'power')
data_dir = data_config.get('path')

print(f"Data config:")
print(f"  Data directory: {data_dir}")
print(f"  Frequency: {freq}")
print(f"  Test end: {test_end}")
print(f"  Target column: {target_col}")

# Load raw data - returns dictionary of DataFrames
print("\n🔄 Loading raw data (dictionary of DataFrames)...")
dfs = preprocessing.get_data(
    data_dir=data_dir,
    config=config,
    freq=freq,
    features=features
)

print(f"✓ Loaded {len(dfs)} DataFrame(s):")
for key, df in dfs.items():
    print(f"  {key}: {df.shape}")
    print(f"    Date range: {df.index.get_level_values(0).min()} to {df.index.get_level_values(0).max()}")
    print(f"    Columns: {list(df.columns)}")

📊 Building complete dataset...

Data config:
  Data directory: /mnt/nas/synthetic/wind/wind_hourly_age_20251103
  Frequency: 1h
  Test end: 2025-10-21 00:00:00+00:00
  Target column: power

🔄 Loading raw data (dictionary of DataFrames)...
✓ Loaded 8 DataFrame(s):
  synth_00164.csv: (156912, 11)
    Date range: 2023-07-24 06:00:00+00:00 to 2025-10-22 15:00:00+00:00
    Columns: ['wind_speed_h78_1', 'wind_speed_h127_1', 'wind_speed_h184_1', 'power', 'park_age', 'cut_in', 'cut_out', 'rated_wind_speed', 'hub_height', 'rotor_diameter', 'altitude']
  synth_03362.csv: (156912, 11)
    Date range: 2023-07-24 06:00:00+00:00 to 2025-10-22 15:00:00+00:00
    Columns: ['wind_speed_h78_1', 'wind_speed_h127_1', 'wind_speed_h184_1', 'power', 'park_age', 'cut_in', 'cut_out', 'rated_wind_speed', 'hub_height', 'rotor_diameter', 'altitude']
  synth_03631.csv: (156912, 11)
    Date range: 2023-07-24 06:00:00+00:00 to 2025-10-22 15:00:00+00:00
    Columns: ['wind_speed_h78_1', 'wind_speed_h127_1', 'wind_sp

## 6. Prepare Train and Test Data

Process each DataFrame through the pipeline and combine, analog to `train_cl.py`.

In [54]:
print("🔧 Processing data through pipeline...\n")

# Use config values as-is (same as during training!)
test_start = pd.Timestamp(config['data']['test_start'], tz='UTC')
test_end = pd.Timestamp(config['data']['test_end'], tz='UTC')
train_frac = config['data']['train_frac']

print(f"Dataset split (from config):")
print(f"  Test start: {test_start}")
print(f"  Test end: {test_end}")
print(f"  Train fraction: {train_frac}")

# Get model parameters
model_config = config.get('model', {})
output_dim = model_config.get('output_dim', 48)
lookback = model_config.get('lookback', 48)
horizon = model_config.get('horizon', 48)
step_size = model_config.get('step_size', 1)

print(f"\nModel parameters:")
print(f"  Output dim: {output_dim}")
print(f"  Lookback: {lookback}")
print(f"  Horizon: {horizon}")
print(f"  Step size: {step_size}")

# Fit global scaler on TRAINING data only (analog to train_cl.py)
print("\n📐 Fitting global scaler on training data only...")
global_scaler_x = StandardScaler()

for key, df in tqdm(dfs.items(), desc="Fitting Global Scaler"):
    df_temp = df.copy()

    # For non-TFT models, create lag features before fitting scaler
    if config['model']['name'] not in ['tft', 'stemgnn']:
        for col in features['observed']:
            all_observed_cols = [new_col for new_col in df_temp.columns if col in new_col]
            for new_col in all_observed_cols:
                df_temp = preprocessing.lag_features(
                    data=df_temp,
                    lookback=lookback,
                    horizon=horizon,
                    lag_in_col=config['data']['lag_in_col'],
                    target_col=new_col
                )
                # Drop the original column if it's not the target and not known
                if new_col != target_col and new_col not in features['known']:
                    df_temp.drop(new_col, axis=1, inplace=True, errors='ignore')

    # Drop target column
    if target_col in df_temp.columns:
        df_temp.drop(target_col, axis=1, inplace=True)

    # Split data to get ONLY training portion for scaler fitting
    t_0 = 0 if config['eval']['eval_on_all_test_data'] else config['eval']['t_0']
    df_train, _ = preprocessing.split_data(
        data=df_temp,
        train_frac=train_frac,
        test_start=test_start,
        test_end=test_end,
        t_0=t_0
    )
    global_scaler_x.partial_fit(df_train)

    del df_temp, df_train
    gc.collect()

print("✓ Global scaler fitted on training data")

# Process data analog to train_cl.py
print("\n🔄 Creating data generator and combining datasets...")
data_generator = tools.create_data_generator(dfs, config, features, scaler_x=global_scaler_x)
X_train, y_train, X_test, y_test, test_data = tools.combine_datasets_efficiently(data_generator)

# Log shapes
if isinstance(X_train, dict):
    if 'known' in X_train:
        print(f'\n📊 Data shapes:')
        print(f'  X_train known: {X_train["known"].shape}, X_test known: {X_test["known"].shape}')
    if 'static' in X_train:
        print(f'  X_train static: {X_train["static"].shape}, X_test static: {X_test["static"].shape}')
    print(f'  X_train observed: {X_train["observed"].shape}, X_test observed: {X_test["observed"].shape}')
    print(f'  y_train: {y_train.shape}, y_test: {y_test.shape}')
else:
    print(f'\n📊 Data shapes:')
    print(f'  X_train: {X_train.shape}, X_test: {X_test.shape}')
    print(f'  y_train: {y_train.shape}, y_test: {y_test.shape}')

print(f"\n✓ Per-park test data available:")
for park_key in test_data.keys():
    print(f"  {park_key}")

gc.collect()

🔧 Processing data through pipeline...

Dataset split (from config):
  Test start: 2025-08-01 00:00:00+00:00
  Test end: 2025-10-21 00:00:00+00:00
  Train fraction: 1

Model parameters:
  Output dim: 48
  Lookback: 48
  Horizon: 48
  Step size: 48

📐 Fitting global scaler on training data only...


Fitting Global Scaler: 100%|██████████| 8/8 [00:05<00:00,  1.56it/s]


✓ Global scaler fitted on training data

🔄 Creating data generator and combining datasets...

📊 Data shapes:
  X_train known: (23336, 96, 3), X_test known: (2592, 96, 3)
  X_train static: (23336, 7), X_test static: (2592, 7)
  X_train observed: (23336, 48, 1), X_test observed: (2592, 48, 1)
  y_train: (23336, 48), y_test: (2592, 48)

✓ Per-park test data available:
  synth_00164.csv
  synth_03362.csv
  synth_03631.csv
  synth_07370.csv
  synth_02638.csv
  synth_02932.csv
  synth_06163.csv
  synth_07374.csv


0

## 7. Load Model and Generate Predictions

In [55]:
print("🤖 Loading trained model...\n")

# Set feature_dim in config (required for model instantiation)
config['model']['feature_dim'] = tools.get_feature_dim(X=X_train)
print(f"Feature dimensions: {config['model']['feature_dim']}")

# Get device
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"\nUsing device: {device}")

# Load hyperparameters based on user selection
if use_hpo_hyperparameters:
    # Try to load from Optuna study
    study_name_suffix = '_'.join(extract_config_name_from_model(selected_model)[0].split('_')[1:])
    study_name = f'cl_m-{model_architecture}_out-{output_dim}_freq-{freq}_{study_name_suffix}'
    print(f"\n📚 Attempting to load hyperparameters from study: {study_name}")

    study = hpo.load_study(config['hpo']['studies_path'], study_name)
    hyperparameters = hpo.get_hyperparameters(config=config, study=study)
    print("✓ Loaded hyperparameters from Optuna study")
    print(f"  hidden_dim: {hyperparameters.get('hidden_dim', 'N/A')}")
    print(f"  n_heads: {hyperparameters.get('n_heads', 'N/A')}")
else:
    # Use standard hyperparameters from config
    hyperparameters = hpo.get_hyperparameters(config=config, study=None)
    print(f"\n📋 Using standard hyperparameters from config")
    print(f"  hidden_dim: {hyperparameters.get('hidden_dim', 'N/A')}")
    print(f"  n_heads: {hyperparameters.get('n_heads', 'N/A')}")

# Load model checkpoint
model_path = os.path.join(models_dir, selected_model)
checkpoint = torch.load(model_path, map_location=device)

# Checkpoint is just state_dict (no metadata)
model_state = checkpoint

# Remove '_orig_mod.' prefix if model was trained with torch.compile()
if any(k.startswith('_orig_mod.') for k in model_state.keys()):
    print("📝 Cleaning state_dict (removing _orig_mod. prefix from compiled model)")
    model_state = {k.replace('_orig_mod.', ''): v for k, v in model_state.items()}

# Create model instance with selected hyperparameters
print(f"\n🏗️ Creating {model_architecture.upper()} model...")
model = models.get_model(config, hyperparameters)
model.load_state_dict(model_state)
model.to(device)
model.eval()

print("✓ Model loaded successfully!")
print(f"  Parameters: {sum(p.numel() for p in model.parameters()):,}")

🤖 Loading trained model...

Feature dimensions: {'observed_dim': 1, 'known_dim': 3, 'static_dim': 7}

Using device: cuda

📋 Using standard hyperparameters from config
  hidden_dim: 16
  n_heads: 4
📝 Cleaning state_dict (removing _orig_mod. prefix from compiled model)

🏗️ Creating TFT model...
✓ Model loaded successfully!
  Parameters: 36,407


## 8. Generate Predictions for All Parks

In [56]:
print("\n🔮 Generating predictions for all parks...\n")

# Store predictions per park
predictions_per_park = {}

with torch.no_grad():
    for park_key, (X_test_park, y_test_park, index_test_park, scaler_y_park) in test_data.items():

        if park_key != target_key:
            continue

        print(f"Processing park: {park_key}")

        # Get predictions
        y_true, y_pred = tools.get_y(
            X_test=X_test_park,
            y_test=y_test_park,
            model=model,
            scaler_y=scaler_y_park,
            device=device
        )

        # Store predictions
        predictions_per_park[park_key] = {
            'X': X_test_park,
            'y_true': y_test_park,
            'y_pred': y_pred,
            'index': index_test_park,
            'scaler_y': scaler_y_park,
            'raw_df': dfs[park_key]
        }

        print(f"  ✓ Predictions shape: {y_pred.shape}")
        print(f"  ✓ Index range: {index_test_park.min()} to {index_test_park.max()}\n")

print(f"✅ Predictions generated for {len(predictions_per_park)} park(s)")


🔮 Generating predictions for all parks...

Processing park: synth_00164.csv
  ✓ Predictions shape: (324, 48)
  ✓ Index range: 2025-08-01 06:00:00+00:00 to 2025-10-20 15:00:00+00:00

✅ Predictions generated for 1 park(s)


## 9. Evaluation

Evaluation of the models is done in the following steps:

In [60]:
print("📊 Calculating metrics by forecast hour...\n")

# Get test indices to group by hour
test_indices = test_data[target_key][2]

# Group predictions by forecast hour (from starttime)
metrics_by_hour = {}

for hour in ['06:00', '09:00', '12:00', '15:00']:
    # Find indices for this hour
    hour_mask = [idx.strftime('%H:%M') == hour for idx in test_indices]

    if not any(hour_mask):
        continue

    # Get predictions and actuals for this hour
    y_true_hour = y_true[hour_mask].flatten()
    y_pred_hour = y_pred[hour_mask].flatten()

    # Calculate metrics
    r2 = r2_score(y_true_hour, y_pred_hour)
    rmse = np.sqrt(mean_squared_error(y_true_hour, y_pred_hour))
    mae = mean_absolute_error(y_true_hour, y_pred_hour)

    metrics_by_hour[hour] = {'R²': r2, 'RMSE': rmse, 'MAE': mae}

# Calculate overall mean
all_metrics = list(metrics_by_hour.values())
mean_r2 = np.mean([m['R²'] for m in all_metrics])
mean_rmse = np.mean([m['RMSE'] for m in all_metrics])
mean_mae = np.mean([m['MAE'] for m in all_metrics])

metrics_by_hour['Mean'] = {'R²': mean_r2, 'RMSE': mean_rmse, 'MAE': mean_mae}

# Create DataFrame for nice display
metrics_df = pd.DataFrame(metrics_by_hour).T
print(metrics_df.to_string())
print("\n✓ Metrics calculated")

📊 Calculating metrics by forecast hour...

             R²      RMSE       MAE
06:00  0.604689  0.056760  0.031821
09:00  0.616284  0.055950  0.031824
12:00  0.586380  0.058100  0.032348
15:00  0.586889  0.058067  0.032213
Mean   0.598561  0.057219  0.032052

✓ Metrics calculated


In [62]:
print("📈 Creating forecast plots...\n")

# Create output directory
model_name = selected_model.replace('.pt', '')
output_dir = f'figs/{model_name}'
os.makedirs(output_dir, exist_ok=True)

# Get the original dataframe for wind speeds
park_df = dfs[park_key]

# Load turbine wind speed from synth CSV
print("Loading turbine wind speeds from synth data...")
park_id = park_key.replace('synth_', '').replace('.csv', '')
synth_path = os.path.join(config['data']['path'], f'synth_{park_id}.csv')

# Load synth data with timestamp as index
synth_df = pd.read_csv(synth_path, sep=';')
synth_df['timestamp'] = pd.to_datetime(synth_df['timestamp'])
synth_df.set_index('timestamp', inplace=True)

# Keep only wind_speed column
turbine_wind = synth_df[['wind_speed']].copy()
print(f"Loaded {len(turbine_wind)} turbine wind speed values")

# Find wind speed columns (NWP with 'h')
wind_cols_nwp = [col for col in park_df.columns if 'wind_speed' in col and '_h' in col]
wind_cols_nwp = sorted(wind_cols_nwp)[:3]  # Take first 3

# Extract heights from column names for legend
nwp_heights = []
for col in wind_cols_nwp:
    match = re.search(r'_h(\d+)', col)
    if match:
        nwp_heights.append(match.group(1))

print(f"NWP wind columns to plot: {wind_cols_nwp}")
print(f"Heights: {nwp_heights}")

# Iterate over all test samples
n_samples = len(y_true)
print(f"Generating {n_samples} plots...")

for i in range(n_samples):
    # Get starttime for this forecast
    starttime = test_indices[i]

    # Get predictions and actuals
    y_true_sample = y_true[i]
    y_pred_sample = y_pred[i]

    # Get wind speeds from original dataframe (NWP forecasts)
    starttime_mask = park_df.index.get_level_values('starttime') == starttime
    wind_data_nwp = park_df.loc[starttime_mask, wind_cols_nwp]

    # Get timestamps for this forecast to fetch turbine wind speeds
    forecast_timestamps = park_df.loc[starttime_mask].index.get_level_values('timestamp')

    # Fetch turbine wind speeds for these timestamps
    wind_data_turbine = []
    for ts in forecast_timestamps:
        if ts in turbine_wind.index:
            wind_data_turbine.append(turbine_wind.loc[ts, 'wind_speed'])
        else:
            wind_data_turbine.append(np.nan)

    # Create figure with two y-axes
    fig, ax1 = plt.subplots(figsize=(14, 7), dpi=300)
    ax2 = ax1.twinx()

    # X-axis: t+1 to t+48
    x = np.arange(1, 49)

    # Left y-axis: Power (y_true and y_pred)
    ax1.plot(x, y_true_sample, 'k-', linewidth=2.5, label='True', alpha=0.9)  # Black
    ax1.plot(x, y_pred_sample, 'b-', linewidth=2, label='Predicted', alpha=0.8)
    ax1.set_xlabel('Forecast Horizon (hours)', fontsize=13)
    ax1.set_ylabel('Power (normalized)', fontsize=13, color='black')
    ax1.tick_params(axis='y', labelcolor='black')
    ax1.set_xticks(np.arange(0, 49, 6))  # 6-hour steps
    ax1.legend(loc='upper left', fontsize=11)
    ax1.grid(True, alpha=0.3)

    # Right y-axis: Wind speeds
    colors_nwp = ['#ff7f0e', '#ff9f4e', '#ffbf7e']  # Orange gradient

    # Plot NWP wind speeds (dashed)
    for idx, (col, color, height) in enumerate(zip(wind_cols_nwp, colors_nwp, nwp_heights)):
        if len(wind_data_nwp) == 48:
            ax2.plot(x, wind_data_nwp[col].values, '--', color=color,
                    linewidth=1.8, label=f'NWP {height}m', alpha=0.75)

    # Plot turbine hub height wind speed (solid red)
    if len(wind_data_turbine) == 48:
        ax2.plot(x, wind_data_turbine, '-', color='#7f7f7f',
                linewidth=2.2, label='Measured 10m', alpha=0.95)

    ax2.set_ylabel('Wind Speed (m/s)', fontsize=13, color='#ff7f0e')
    ax2.tick_params(axis='y', labelcolor='#ff7f0e')
    ax2.legend(loc='upper right', fontsize=10)

    # Title
    plt.title(f'Forecast starting: {starttime.strftime("%Y-%m-%d %H:%M")}',
             fontsize=15, fontweight='bold')
    plt.tight_layout()

    # Save figure
    filename = starttime.strftime("%Y-%m-%d %H_%M_%S") + '.png'
    filepath = os.path.join(output_dir, filename)
    plt.savefig(filepath, dpi=300, bbox_inches='tight')
    plt.close()

    # Progress indicator
    if (i + 1) % 50 == 0:
        print(f"  Generated {i+1}/{n_samples} plots...")

print(f"\n✓ All plots saved to: {output_dir}/")

📈 Creating forecast plots...

Loading turbine wind speeds from synth data...
Loaded 20016 turbine wind speed values
NWP wind columns to plot: ['wind_speed_h127_1', 'wind_speed_h184_1', 'wind_speed_h78_1']
Heights: ['127', '184', '78']
Generating 324 plots...
  Generated 50/324 plots...
  Generated 100/324 plots...
  Generated 150/324 plots...
  Generated 200/324 plots...
  Generated 250/324 plots...
  Generated 300/324 plots...

✓ All plots saved to: figs/cl_m-tft_out-48_freq-1h_wind_eight_static/
